In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [2]:
from transformers import RobertaTokenizer, TFRobertaModel, BertTokenizer, TFBertModel

In [3]:
df = pd.read_csv("../efcamdat_test_data.csv")

In [4]:
texts = df['text'].to_numpy()
true_labels = df['cefr_numeric'].apply(lambda x: int(x) - 1).to_numpy()

In [5]:
bert_model = tf.keras.models.load_model('../models/bert_model_final.keras', {'TFBertModel': TFBertModel})
roberta_cl_model = tf.keras.models.load_model('../models/roberta_model_final.keras', {'TFRobertaModel': TFRobertaModel})
#roberta_gru_model = tf.keras.models.load_model('../models/roberta_gru_model.keras', {'TFRobertaModel': TFRobertaModel})

Metal device set to: Apple M1 Pro
Your GPU may run slowly with dtype policy mixed_float16 because it does not have compute capability of at least 7.0. Your GPU:
  METAL, no compute capability (probably not an Nvidia GPU)
See https://developer.nvidia.com/cuda-gpus for a list of GPUs and their compute capabilities.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


2024-07-25 18:33:30.838014: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-07-25 18:33:30.838166: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

MAX_SEQUENCE_LENGTH = 85

bert_tokens = bert_tokenizer([str(x) for x in texts], truncation=True, padding='max_length', max_length=MAX_SEQUENCE_LENGTH, return_tensors='tf')
roberta_tokens = roberta_tokenizer([str(x) for x in texts], truncation=True, padding='max_length', max_length=MAX_SEQUENCE_LENGTH, return_tensors='tf')

In [7]:
bert_dataset = tf.data.Dataset.from_tensor_slices((
        {
            'input_ids': bert_tokens['input_ids'],
            'token_type_ids': bert_tokens['token_type_ids'],
            'attention_mask': bert_tokens['attention_mask']
        },
        true_labels
    ))

roberta_dataset = tf.data.Dataset.from_tensor_slices((
        {
            'input_ids_layer': roberta_tokens['input_ids'],
            'attention_mask_layer': roberta_tokens['attention_mask']
        },
        true_labels
    ))

bert_dataset = bert_dataset.batch(32)
roberta_dataset = roberta_dataset.batch(32)

In [8]:
bert_preds = bert_model.predict(bert_dataset)
roberta_cl_preds = roberta_cl_model.predict(roberta_dataset)
#roberta_gru_preds = roberta_gru_model.predict(roberta_dataset)

2024-07-25 18:34:15.707899: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-07-25 18:34:15.709809: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1097/1097 [==============================] - 309s 280ms/step


2024-07-25 18:39:25.568760: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


   1/1097 [..............................] - ETA: 1:09:19

2024-07-25 18:39:28.430644: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1097/1097 [==============================] - 315s 284ms/step


In [9]:
bert_preds = np.argmax(bert_preds, axis=-1)
roberta_cl_preds = np.argmax(roberta_cl_preds, axis=-1)

In [10]:
bert_wrong_ind = np.where(bert_preds != true_labels)
rob_wrong_ind = np.where(roberta_cl_preds != true_labels)

In [48]:
bert_wrong_texts = texts[bert_wrong_ind]
rob_wrong_texts = texts[rob_wrong_ind]

print(f"BERT has {len(bert_wrong_texts)} wrong predictions")
print(f"RoBERTa has {len(rob_wrong_texts)} wrong predictions")

BERT has 848 wrong predictions
RoBERTa has 1133 wrong predictions


## Exploring Texts that Both Got Wrong

In [49]:
both_wrong = np.intersect1d(rob_wrong_ind, bert_wrong_ind)
print(f"There are {len(both_wrong)} predictions that both got wrong")

There are 427 predictions that both got wrong


In [50]:
#value counts of the labels they got wrong the most 
np.asarray(np.unique(true_labels[both_wrong], return_counts=True)).T

array([[  0,  24],
       [  1,  65],
       [  2, 156],
       [  3,  66],
       [  4, 116]])

In [104]:
bert_only_wrong = [i for i in texts[bert_wrong_ind] if i not in texts[rob_wrong_ind]]
rob_only_wrong = [i for i in texts[rob_wrong_ind] if i not in texts[bert_wrong_ind]]
bert_only_wrongid = [i for i in bert_wrong_ind[0] if i not in rob_wrong_ind[0]]
rob_only_wrongid = [i for i in rob_wrong_ind[0] if i not in bert_wrong_ind[0]]

In [105]:
len(bert_only_wrongid)

421

In [106]:
len(rob_only_wrongid)

706

In [125]:
def sample_both_wrongs(class_n, who='both', verbose=True):
    if who == 'both':
        idxs = np.intersect1d(np.where(true_labels == class_n), both_wrong)
        idx = np.random.choice(idxs)
    elif who == 'bert':
        idxs = np.intersect1d(np.where(true_labels == class_n), bert_only_wrongid)
        idx = np.random.choice(idxs)
    else:
        idxs = np.intersect1d(np.where(true_labels == class_n), rob_only_wrongid)
        idx = np.random.choice(idxs)
    if verbose:
        print("BERT predicted label: ", bert_preds[idx])
        print("RoBERTa predicted label: ", roberta_cl_preds[idx])
        print("Actual label: ", true_labels[idx])
    return bert_preds[idx], roberta_cl_preds[idx], true_labels[idx], texts[idx]

### Sample from A1

In [73]:
sample_both_wrongs(0)

BERT predicted label:  1
RoBERTa predicted label:  1
Actual label:  0


"\n\t  My name is Evelyne and I am retired of the work about since months ago. Now I get up at 8 for looking my harder and the birds. I go walking my my pretty dog, Twiny. It's my beloved dog ! So I like gardening and i go walking  2 afernoon in the week with a club. Also I like take photo and my hobby is scrapbooking.\n\t"

In [75]:
sample_both_wrongs(0)

BERT predicted label:  2
RoBERTa predicted label:  1
Actual label:  0


"\n\t  Hi! I don't really care about what everyone likes or don't likes. I'm a passionate cook so you better like it. Everything will tastes delicious. I guess you will agree with me. I hope all off you are free and I'm looking forward to meeting you next Friday. See you.\n\t"

### Sample from A2

In [76]:
sample_both_wrongs(1)

BERT predicted label:  3
RoBERTa predicted label:  3
Actual label:  1


"\n\t  Dear Polaris,   buy a good book about history which has more than two thousands sides. You should need some weeks to go through it. After that you will forget all the scrap you were looking for. Believe me, after a good book about history there will remain only one whish: a time machine. But you won't find a shop on this entire planet where you could buy such a machine. Kind Regards,  Tom\n\t"

In [77]:
sample_both_wrongs(1)

BERT predicted label:  2
RoBERTa predicted label:  2
Actual label:  1


"\n\t  -Welcome to Domenici's Family Blog- In this Blog you will found some resume about the members of my family and what each like to do and eat or drink. Danilo Domenicis (It's Me) We can say that's Danilo is a kind of person that like to workout and he works very much! He is 21 yo and works with IT. He loves to eat Pizza but cause he likes also to workout is normally to see him eating just chicken and potatoe. Silvana Domenicis (Danilo's Mother) Silvana is a good person that work a lot to keep all the bills and organization in Domenici's house. She loves to eat pasta and she normally likes to go to the movie.\n\t"

### Sample from B1

In [78]:
sample_both_wrongs(2)

BERT predicted label:  3
RoBERTa predicted label:  3
Actual label:  2


"\n\t  The art of deal unfortunately I don't have for purchase,  but I always make negotiation in my personal life, for example what movie we'll watch, places to visit and dates to do all these. I almost never negotiate any purchase, It is work for my father, he is a master in negotiations! I can't because sellers have strategies that I can't escape, so I buy by pressure and no discount! However I have learned negotiations and now I am firmer to buy anything. I've already worked in negotiations, when I was eighteen to nineteen years old, It was for a banking and I had great argumentations, counteroffer and solutions to offer for the costumer. I could to explore the reason of dissatisfaction with the bank and I started to negotiation with them.\n\t"

In [80]:
sample_both_wrongs(2)

BERT predicted label:  3
RoBERTa predicted label:  3
Actual label:  2


"\n\t  Good manners in my country Brazil is a beautiful country visited by millions of people. Here are some basic rules of etiquettes when you're eating at a restaurant or in a home's friend: Firstly, you're supposed to put the napkin on his lap. Do not put your elbows on the table. Chew food with mouth closed and slouly. Do not overfull the fork or spoon. Do not blow the soup to cool. Do not slurping when drinking liquids or soups. Do not smoke in prohibited places. Avoid talking with your mouth full of food. As others countries, You're expected do not spitting in the streets. Staring and pointing someone are considered offensive.\n\t"

### Sample from B2

In [81]:
sample_both_wrongs(3)

BERT predicted label:  2
RoBERTa predicted label:  2
Actual label:  3


"\n\t  Hello Tom. Do you remember that old friend from claim her about the hug she gave me moments ago. I don't know what to do, you know she isn't my girlfriend but how to explain besides I couldn't say a word maybe it was the impression, I didn't know, but what an awkward situation.\n\t"

In [82]:
sample_both_wrongs(3)

BERT predicted label:  2
RoBERTa predicted label:  2
Actual label:  3


"\n\t  I think that the internet really change the world.For instance,if we would like to go somewhere We've never been to,we should confirm how to get there by searching a map before the launch of the internet.We can easily search anything we want to know by using the internet.But using the internet is the double-edged sword indeed.The internet always gives us a lot of information,so we tend to use the information in a casual way rather than tend to think by ourselves.For instance,there were large social problems that many Japanese college students submit their graduation thesis whose conclusions are gathered by the internet.The internet can enlarge our vision but may deprive us of thoughtfulness and originality.We have to know its benefit and deficit,then we have to make good use of it,I think.\n\t"

### Sample from C1

In [83]:
sample_both_wrongs(4)

BERT predicted label:  2
RoBERTa predicted label:  2
Actual label:  4


"\n\t  People's lives will be much easier in 20 years thanks to technical improvement.  Retinal scanning will facilitate the identification process. No passports, no visas, no administrative procedures will warrantee a comfortable life.  Artificial intelligence will be in the run to replace people in some positions like tourist guide, receptionist, assistant, security guard...There are certainly less jobs, but living conditions will be much better.  People can work at home or anywhere as they wish through Internet and telebeaming. With the help of satellite cellphones, communication will be thus far effective. Transport will be different, flying trains, high speed bus which are hydropowered are public transport means. There will be no cars which use fuel. The pollution will consequently diminished. Houses will be built ecofriendly, with more green space and lightened by solar-powered bulbs. Waste will be obligated to be reused or recycled. Plastic material will be highly taxed and bann

In [84]:
sample_both_wrongs(4)

BERT predicted label:  2
RoBERTa predicted label:  2
Actual label:  4


"\n\t  I'd like to show what are similar or different between Japanese and Hawaiian elementary schools. Because I grew up in Hawaii and went back to Japan when I was 15 years old. The first different thing is teachers. In Japan, a teacher has his/her classroom and teaches all subjects. Students study in the same classroom without going to other rooms. A teacher comes in and goes to the teacher's office after a class. Teachers need to prepare many things and have several skills. On the other hand, students go to each room to take a class in Hawaii. They have English, math, science, PE's teachers and others. Teachers are experts. Each teacher has the ability to teach the subject deeply. The second different thing is how students study subjects in school. In Japan, classes are teacher-oriented. Teachers have an answer and students have to meet it. Otherwise, students fail. There are a lot of time to get information from teachers but less time to practice. On the other hand, classes are st

## Exploring BERT Wrong but RoBERTa Right

### A1

In [108]:
sample_both_wrongs(0, 'bert')

BERT predicted label:  1
RoBERTa predicted label:  0
Actual label:  0


"\n\t  Hi! I'm having a birthday party on sunday. I invite you can go on. In the party, We can meet many friends. In the morning , We can watch TV. In the afternoon, We have a lunch in my house. I can make much delicious food, I think you must like it. Hope you go on.  Johnson\n\t"

### A2

In [109]:
sample_both_wrongs(1, 'bert')

BERT predicted label:  2
RoBERTa predicted label:  1
Actual label:  1


'\n\t  To All staff members.   This office has received many complaints from clients concerning the lack of professional dress in our offices. four example, today I noticed men wearing baggy trousers, basebal caps, and polyester shirts. This reflects poorly on our company and is not the trend for a company dealing with bankers and stock brokers.   We want you to wear stylish clothes, but we do not want to see our staff following trends of teenagers and gang members.  \n\t'

### B1

In [110]:
sample_both_wrongs(2, 'bert')

BERT predicted label:  1
RoBERTa predicted label:  2
Actual label:  2


'\n\t  At first I met my husband in elementary school. He was in class two and I in class one. He was very tall and he had a terrible suitcase instead of a satchel.  A few years later I was sixteen years old, I went to the bowling with my friends.  We went there with motorbike. One of my friends brought his best friend. He brought me home at 23 oclock with his motorbike. It was winter and it was very cold. But we stood at the door and talked all night. When it was five oclock in the Morning, we realized that are invited to the same party today. He brought me home again and we talked all night. He was very shy, but now I took all my courage and kissed him. This was the beginning of our relationship.\n\t'

### B2

In [111]:
sample_both_wrongs(3, 'bert')

BERT predicted label:  4
RoBERTa predicted label:  3
Actual label:  3


"\n\t  Although working together for so many years, it still startles me how Nelson begins each and every day: always reinvigorated and prompt for any possible challenges he will face. Nelson is, as a rule, an affable and approachable guy. Nevertheless, any subtle constructive criticism devastates him. Let's face it: we can't always walk on egg shells around him if we want the work team to accomplish its demanding goals. Nelson excels at communicating difficult concepts to grasp by most of us in easy to understand daily language. His communication skills have skyrocketed in the last three years. He is undoubtedly a creative and competent guy but has a conspicuous tendency to act before thinking, therefore, failing to create new solutions. Nelson is a loyal and trustworthy employee, who always gets the job perfectly done and whose general performance is above expectations.\n\t"

### C1

In [112]:
sample_both_wrongs(4, 'bert')

BERT predicted label:  1
RoBERTa predicted label:  4
Actual label:  4


"\n\t  The best website where to buy on in Italy is Mondatori.it.. I am custom to spend a lot of money there. Is very simple and easy to use. Is rich on information and the customer service is at your disposal 24/7. A vast amount of customers are online and they exchange information using a chat pod. The secret of this website? The more you spend the more discount you'll have on your next purchasing. This is due to the large quantity of goods that they possess and in this way they are much more competitive than a small shop in town. I always decide to buy more than one article and reach 250 euro as total amount. In this way, the shipping taxes are free. What do I buy on Mondadori.it? You can find anything you want. Starting from music, books, clothes and IT devices. As I said, everything is online, and the price is unbelievably favorable. There is also a vacation section that permits you to dream about your next vacation, and the reason why a lot of us decide to buy there instead of tr

## Exploring RoBERTa Wrong but BERT Right

### A1

In [113]:
sample_both_wrongs(0,'roberta')

BERT predicted label:  0
RoBERTa predicted label:  2
Actual label:  0


'\n\t  Hi, Ed. Let me thing... I have 10 dollars too. Meaby we can buy special pens, I remember saw some one in the mall, those pens have a special form, its for a better writing. Well, tell me if you are agre. Sosa.\n\t'

### A2

In [114]:
sample_both_wrongs(1, 'roberta')

BERT predicted label:  1
RoBERTa predicted label:  2
Actual label:  1


'\n\t  Dear, mrs. I am looking for a job, so my experience it not much, after I work the waitres but i need more money. but I studiend now at the university so i see you need help marketing assitant. maybe if you need any call me. ok thanks you sinceraly Emilio\n\t'

### B1

In [115]:
sample_both_wrongs(2, 'roberta')

BERT predicted label:  2
RoBERTa predicted label:  3
Actual label:  2


"\n\t  Two moth ago I bougth my first tablet, before I had bougth I did not like of idea of have a tablet becase I didn't see useful for this. But today I am very happy with my new mobile gadget. It goes with me in all places and I can access the internet conection in everwhere. This tablet make me more productive and also I can talk with my friends and family. Other thing important is the games, I can buy many kinds of games and it is an excelent thing for make when I don't have anything for to make. An thing that I use much is the calendar on tablet, it help me to remenber my mettings and my tasks.\n\t"

### B2

In [116]:
sample_both_wrongs(3, 'roberta')

BERT predicted label:  3
RoBERTa predicted label:  4
Actual label:  3


'\n\t  Dear colleagues,  as project manager I want to give you a comprehensive induction to our new and challengening project which will start next month. On the next page you can inform you about all the details of this project as well as terms and conditons.  But first of all I propose some importants aspects of our corporate management strategy which is absolutely necessary in order to process all tasks successfully. Every second day in the week we will have a meeting about one hour. The most important issue in these meetings is communcation. Please notice, before embarking on a strategy we initiate a brainstorming which should help all project members and make the goals of our team clear and incisive.  Moreover, everyone should be prepared and report about problems and challenges of current tasks. Please remember to turn off mobile phones and any other potential disturbances.  In conclusion you have to know that the project should be finished within three months. Our customer is fo

### C1

In [117]:
sample_both_wrongs(4, 'roberta')

BERT predicted label:  4
RoBERTa predicted label:  2
Actual label:  4


"\n\t  Dear Mr Freeman, I want you to know that we are working hard to fulfil your order before the scheduled date. However, I want to be sure about the color we will be using.  So, I'm sending you some samples for your apreciation. Please let me known which one you prefer. I need to receive your feedback by October 21. I also would like to invite you to visit our factory on the first week of November, when you could see our progress and we could discuss the packaging options. Please let me know if you agree with that and what is the date that suits you most. Best regards, Lucila\n\t"

## Seeing if we can explain why certain texts were wrong for each model

In [121]:
# Code from baseline model metrics 
import pandas as pd
import nltk
import spacy
import textstat
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Download NLTK data
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Load Spacy model
nlp = spacy.load('en_core_web_sm')

# Define feature extraction functions
def word_count(text):
    return len(nltk.word_tokenize(text))

def syllable_count(text):
    return textstat.syllable_count(text)

def character_count(text):
    return len(text)

def complex_word_count(text):
    return textstat.lexicon_count(text, removepunct=True) - textstat.difficult_words(text)

def vocab_size(text):
    return len(set(nltk.word_tokenize(text)))

def lexical_diversity(text):
    words = nltk.word_tokenize(text)
    return len(set(words)) / len(words)

def noun_chunks(text):
    doc = nlp(text)
    return len(list(doc.noun_chunks))

def flesch_kincaid_score(text):
    return textstat.flesch_kincaid_grade(text)

def dale_chall_score(text):
    return textstat.dale_chall_readability_score(text)

def gunning_fog_index(text):
    return textstat.gunning_fog(text)

def coleman_liau_index(text):
    return textstat.coleman_liau_index(text)

def automated_readability_index(text):
    return textstat.automated_readability_index(text)

# create df with features
def extract_features(df, text_column):
    features = pd.DataFrame()
    features['Word Count'] = df[text_column].apply(word_count)
    features['Syllable Count'] = df[text_column].apply(syllable_count)
    features['Character Count'] = df[text_column].apply(character_count)
    features['Complex Word Count'] = df[text_column].apply(complex_word_count)
    features['Vocab Size'] = df[text_column].apply(vocab_size)
    features['Lexical Diversity'] = df[text_column].apply(lexical_diversity)
    features['Noun Chunks'] = df[text_column].apply(noun_chunks)
    features['Flesch Kincaid Score'] = df[text_column].apply(flesch_kincaid_score)
    features['Dale Chall Score'] = df[text_column].apply(dale_chall_score)
    features['Gunning Fog Index'] = df[text_column].apply(gunning_fog_index)
    features['Coleman Liau Index'] = df[text_column].apply(coleman_liau_index)
    features['Automated Readability Index'] = df[text_column].apply(automated_readability_index)
    return features


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/danielskahill/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/danielskahill/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [160]:
def generate_wrong_metrics_df(model):
    classes = [0, 1, 2, 3, 4]
    df = pd.DataFrame(columns=['bert_pred', 'roberta_pred', 'true', 'text'])
    for c in classes:
        for i in range(24):
            b, r, t, text = sample_both_wrongs(c, model, verbose=False)
            temp = pd.DataFrame({'bert_pred': [b], 'roberta_pred': [r], 'true': [t], 'text': [text]})
            df = pd.concat([df, temp])
    df = df.reset_index()
    
    features = extract_features(df, 'text')
    features = features.reset_index()
    merged = pd.merge(df, features, left_index=True, right_index=True)
    features['label'] = merged['true']
    return merged, features
    

In [161]:
roberta_df, roberta_features = generate_wrong_metrics_df('roberta')

In [162]:
bert_df, bert_features = generate_wrong_metrics_df('bert')

In [167]:
rob_wrong_stats = roberta_features.groupby('label').agg('mean')
rob_wrong_stats 

,index,Word Count,Syllable Count,Character Count,Complex Word Count,Vocab Size,Lexical Diversity,Noun Chunks,Flesch Kincaid Score,Dale Chall Score,Gunning Fog Index,Coleman Liau Index,Automated Readability Index
label,,,,,,,,,,,,,
0,11.5,67.208333,68.833333,293.458333,46.666667,47.291667,0.710995,18.416667,4.591667,8.258750,6.705833,5.340417,5.604167
1,35.5,78.041667,85.291667,350.750000,58.416667,52.916667,0.688758,22.000000,4.908333,7.410000,6.512917,5.194583,5.133333
2,59.5,132.458333,158.333333,638.333333,98.833333,85.833333,0.653122,33.625000,8.141667,8.101667,10.252917,7.347500,9.358333
3,83.5,164.166667,203.583333,810.375000,126.333333,95.333333,0.585100,45.041667,8.258333,7.544167,10.253333,7.490000,9.291667
4,107.5,185.125000,229.916667,915.625000,139.083333,108.750000,0.591990,47.333333,8.037500,7.929583,10.106667,8.121250,9.162500


In [168]:
bert_wrong_stats = bert_features.groupby('label').agg('mean')
bert_wrong_stats 

,index,Word Count,Syllable Count,Character Count,Complex Word Count,Vocab Size,Lexical Diversity,Noun Chunks,Flesch Kincaid Score,Dale Chall Score,Gunning Fog Index,Coleman Liau Index,Automated Readability Index
label,,,,,,,,,,,,,
0,11.5,55.500000,57.541667,243.166667,42.833333,37.958333,0.687275,15.541667,2.900000,6.295417,4.695000,3.884583,3.229167
1,35.5,76.000000,89.333333,361.458333,51.375000,51.458333,0.685836,20.291667,7.591667,8.812917,9.763750,7.345000,8.570833
2,59.5,106.791667,118.833333,498.208333,80.750000,70.208333,0.658725,29.250000,5.229167,7.376667,7.239583,6.082083,6.262500
3,83.5,154.666667,183.416667,730.666667,115.583333,92.083333,0.612994,40.500000,7.029167,7.735417,9.369167,6.840417,7.616667
4,107.5,191.000000,239.416667,956.791667,141.666667,109.958333,0.583796,49.166667,9.579167,8.289167,11.692917,8.684167,11.437500


In [169]:
rob_wrong_stats - bert_wrong_stats 

,index,Word Count,Syllable Count,Character Count,Complex Word Count,Vocab Size,Lexical Diversity,Noun Chunks,Flesch Kincaid Score,Dale Chall Score,Gunning Fog Index,Coleman Liau Index,Automated Readability Index
label,,,,,,,,,,,,,
0,0.0,11.708333,11.291667,50.291667,3.833333,9.333333,0.023721,2.875000,1.691667,1.963333,2.010833,1.455833,2.375000
1,0.0,2.041667,-4.041667,-10.708333,7.041667,1.458333,0.002921,1.708333,-2.683333,-1.402917,-3.250833,-2.150417,-3.437500
2,0.0,25.666667,39.500000,140.125000,18.083333,15.625000,-0.005603,4.375000,2.912500,0.725000,3.013333,1.265417,3.095833
3,0.0,9.500000,20.166667,79.708333,10.750000,3.250000,-0.027894,4.541667,1.229167,-0.191250,0.884167,0.649583,1.675000
4,0.0,-5.875000,-9.500000,-41.166667,-2.583333,-1.208333,0.008194,-1.833333,-1.541667,-0.359583,-1.586250,-0.562917,-2.275000
